# 步骤1：给大模型提供一个可以调用的函数

在这个示例中，假设您希望允许模型调用代码库中的 `get_weather` 函数，该函数接受一个 `city` 并查询对应城市的天气

下面这个函数可以传入如"TianJin"、"HangZhou"、"ShenZhen"等城市名

返回该城市的天气和温度，格式如：{"temperature": 31, "weather_descriptions": ["Sunny"]}


In [2]:
import os
from dotenv import load_dotenv
import requests
import json

def get_weather(city:str):
    # Load environment variables from .env file
    load_dotenv()

    # Get the access key from the environment variables
    access_key = os.getenv('WEATHER_ACCESS_KEY')

    # Define the API URL with the access key
    url = f"https://api.weatherstack.com/current?access_key={access_key}&query=TianJin"

    payload = {}
    headers = {}

    # Make the API request
    response = requests.get(url, headers=headers, data=payload)
    res_json = json.loads(response.text)


    # 创建新的 JSON 对象，只保留 temperature 和 weather_descriptions 两个参数
    new_data = {
        "temperature": res_json["current"]["temperature"],
        "weather_descriptions": res_json["current"]["weather_descriptions"]
    }
    # 将新的 JSON 对象转换为 JSON 字符串
    new_json = json.dumps(new_data)
    
    # Print the response text
    print(new_json)
    return new_json

# get_weather("TianJin")



{"temperature": 31, "weather_descriptions": ["Sunny"]}


'{"temperature": 31, "weather_descriptions": ["Sunny"]}'

# 步骤2：向模型描述函数，以便模型知道如何调用
现在我们清楚了希望模型调用的函数，接下来将创建一个“函数定义”，向模型描述该函数。此定义既说明了函数的功能（以及可能何时应调用它），也指明了调用函数所需的参数。

函数定义的参数部分应使用 JSON Schema 进行描述。当模型生成函数调用时，它将依据您提供的模式生成参数。

在这个示例中，它可能看起来像这样：
``` json
{
    "name": "get_weather",
    "description": "获取某个城市的天气。每当您需要知道天气时，请调用此命令，例如当用户询问“天津今天天气如何”时",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "城市的拼音，如天津市是'TianJin'",
            },
        },
        "required": ["city"],
        "additionalProperties": False,
    }
}

```

# 步骤3：将函数定义作为“工具”传递给模型，同时附上消息
接下来，我们需要在调用聊天补全 API 时，在可用“tools”数组中提供我们的函数定义。

一如既往，我们将提供一系列“消息”，这些消息可能包含您的提示，或者用户与助手之间完整的来回对话。

此示例展示了如何调用聊天补全 API，为处理商店客户咨询的助手提供相关函数和消息。

In [18]:
from openai import OpenAI

def talk_to_assistant(prompt):
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "获取某个城市的天气。每当您需要知道天气时，请调用此命令，例如当用户询问“天津今天天气如何”时",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type": "string",
                            "description": "城市的拼音，如天津市是'TianJin'",
                        },
                    },
                    "required": ["city"],
                    "additionalProperties": False,
                }
            }   
        }
    ]

    messages = [
        {"role": "system", "content": "你是一个乐于助人的客户支持助理。使用提供的工具来帮助用户。"},
        {"role": "user", "content": prompt}
    ]

    client = OpenAI()

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
    )

    print(completion)
    return completion

completion = talk_to_assistant("今天北京的天气如何？")

ChatCompletion(id='chatcmpl-9wjDmhDYIJrpOeJoqETyuZZwrQBT2', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_w8whUQQ5yjqV89zITHl24DKg', function=Function(arguments='{"city":"BeiJing"}', name='get_weather'), type='function')]))], created=1723783882, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_48196bc67a', usage=CompletionUsage(completion_tokens=16, prompt_tokens=111, total_tokens=127))


# 步骤4：接收并处理模型响应
## 如果模型决定不应调用任何函数
如果模型未生成函数调用，则响应将按照 Chat Completions 的常规方式直接回复用户。

例如，在这种情况下，chat_response.choices[0].message 可能包含：

```
chat.completionsMessage(content='Hi there! I can help with that. Can you please provide your order ID?', role='assistant', function_call=None, tool_calls=None)
```

这说明大模型未能从对话中获取调用函数所需的参数，需要进入异常处理流程

## 如果模型生成了一个函数调用
如果模型生成了一个函数调用，它将根据您提供的参数定义生成调用的参数。

咱们步骤三如果执行成功的话，正常的响应是这样的：
```
ChatCompletion(id='chatcmpl-9wi1GmADOIWaZUagaPgvW5DYLFzlW', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_LrGoJZpWmUgc9yoKbwzxHCHa', function=Function(arguments='{"city":"BeiJing"}', name='get_weather'), type='function')]))], created=1723779262, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_507c9469a1', usage=CompletionUsage(completion_tokens=16, prompt_tokens=111, total_tokens=127))
```

咱们挑出里面有用的部分
```
Choice(
	finish_reason='tool_calls', 
	index=0, 
	logprobs=None, 
	message=ChatCompletionMessage(
		content=None, 
		refusal=None, 
		role='assistant', 
		function_call=None, 
		tool_calls=[
			ChatCompletionMessageToolCall(
				id='call_LrGoJZpWmUgc9yoKbwzxHCHa', 
                # 其实最有用的就这一行，其他的是用于判断大模型返回结果是否正常的，各种异常情况需要根据实际业务来处理，咱这里不展开了
				function=Function(arguments='{"city":"BeiJing"}', name='get_weather'), type='function'
			)
		]
	)
)
```

In [31]:
# 通过代码提取有用的数据
def do_function(completion):
    tool_call = completion.choices[0].message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('city')
    print(city)

    function_name = tool_call.function.name
    print(function_name)


    # 使用反射机制动态调用函数
    if hasattr(__import__('__main__'), function_name):
        function_to_call = getattr(__import__('__main__'), function_name)
        result = function_to_call(**arguments)
        print(result)
    else:
        print(f"Function {function_name} not found.")
    return  result

res_function = do_function(completion)

BeiJing
get_weather
{"temperature": 31, "weather_descriptions": ["Partly cloudy"]}
{"temperature": 31, "weather_descriptions": ["Partly cloudy"]}


'{"temperature": 31, "weather_descriptions": ["Partly cloudy"]}'

到这里我们就完成了利用大模型调用函数的功能开发

# 步骤5 将函数调用结果返回给大模型

现在我们在本地执行了函数调用，需要将此次函数调用的结果返回给聊天补全 API，以便模型生成用户应看到的实际响应，下面是一个完整的例子：

In [43]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "获取某个城市的天气。每当您需要知道天气时，请调用此命令，例如当用户询问“天津今天天气如何”时",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "城市的拼音，如天津市是'TianJin'",
                    },
                },
                "required": ["city"],
                "additionalProperties": False,
            }
        }   
    }
]

res_function = do_function(completion)

messages = [
    {"role": "system", "content": "你是一个乐于助人的客户支持助理。使用提供的工具来帮助用户。"},
    {"role": "user", "content": "今天北京天气如何？"},
    {"role": "assistant", "content": '{"temperature": 33, "weather_descriptions": ["Sunny"]}'}
]



client = OpenAI()
# 调用OpenAI API的聊天完成端点，将工具调用结果送回模型
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

print(response)

BeiJing
get_weather
{"temperature": 33, "weather_descriptions": ["Sunny"]}
{"temperature": 33, "weather_descriptions": ["Sunny"]}
ChatCompletion(id='chatcmpl-9wkgPupsPGFo30bvitqmV3oy0geNS', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='今天北京的天气晴朗，气温约为33°C。请注意防晒和保持水分哦！如果你需要更多的信息或建议，请告诉我！', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723789501, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_48196bc67a', usage=CompletionUsage(completion_tokens=35, prompt_tokens=55, total_tokens=90))


得到最终反馈：今天北京的天气晴朗，气温约为33°C。请注意防晒和保持水分哦！如果你需要更多的信息或建议，请告诉我！